In [1]:
import numpy as np
import random
from random import shuffle
import os
import gensim
import re
import pickle
import tensorflow as tf
import operator
import math
import sys
from copy import deepcopy
from collections import Counter
import string
import fileinput
from nltk.tokenize import sent_tokenize, word_tokenize 
import nltk

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import split, explode
from pyspark.sql import functions as func
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors, DenseVector, VectorUDT

from tensorflow.python.ops import rnn, rnn_cell
from tensorflow.python.framework import ops
from tensorflow.models.rnn.translate import seq2seq_model

## Loading Data

In [ ]:
os.environ["PYSPARK_PYTHON"] = 'python2'

In [ ]:
sc = SparkContext('local[4]')
sqlcontext = SQLContext(sc)

In [2]:
data_dir = '/media/ai2-rey/data_disk/data_sets/SQuAD/'

In [3]:
def load_data(data_dir, prefix):
    filenames = [data_dir + prefix + tag for tag in ['.context', '.question', '.answer', '.span']]

    data = []
    column = []
    line_iter = fileinput.input(filenames)
    for line in line_iter:
        if line_iter.isfirstline():
            data.append(column)
            column = []
        column.append(line.rstrip())
    data.append(column)
    line_iter.close()
    data = data[1:]
    return list(zip(data[0], data[1], data[2], data[3]))

In [4]:
train_data = load_data(data_dir, 'train')
train_cont, train_ques, train_ans, train_span = list(zip(*train_data))

val_data = load_data(data_dir, 'val')
val_cont, val_ques, val_ans, val_span = list(zip(*val_data))

test_data = load_data(data_dir, 'dev')
test_cont, test_ques, test_ans, test_span = list(zip(*test_data))

In [5]:
del train_data, val_data, test_data

In [6]:
print(val_cont[120])
print(val_ques[120])
print(val_ans[120])
print(val_span[120].split())

The Martorana church ( decorated around 1143 ) looked originally even more Byzantine although important parts were later demolished . The dome mosaic is similar to that of the Cappella Palatina , with Christ enthroned in the middle and four bowed , elongated angels . The Greek inscriptions , decorative patterns , and evangelists in the squinches are obviously executed by the same Greek masters who worked on the Cappella Palatina . The mosaic depicting Roger II of Sicily , dressed in Byzantine imperial robes and receiving the crown by Christ , was originally in the demolished narthex together with another panel , the Theotokos with Georgios of Antiochia , the founder of the church .
The Martorana church was likely decorated by the same greek masters who decorated which church ?
the Cappella Palatina
['68', '70']


In [7]:
class QA_Dataset:
  
    def __init__(self, context, question, answer, span, word2id_dict=None, 
                 id2word_dict=None, max_vocab_size=None, autopad_context=None, 
                 autopad_ques = None, autopad_answer=None, 
                 context_pad_len=None, ques_pad_len = None, answer_pad_len=None):

        assert autopad_context in {'min', 'max', 'avg'}
        assert autopad_answer in {'min', 'max', 'avg'}
        assert autopad_ques in {'min', 'max', 'avg'}
        
        self.context = context
        self.question = question
        self.answer = answer
        self.span = span
#         span_ids = [i.split() for i in span]
#         t_span = []
#         for s in span_ids:
#             s = [int(c) for c in s]
#             t_span.append(s)
#         self.span = t_span
        
        self.max_vocab_size = max_vocab_size
        self.context_pad_len = context_pad_len
        self.ques_pad_len = ques_pad_len
        self.answer_pad_len = answer_pad_len
        self.__autopad_context = autopad_context
        self.__autopad_ques = autopad_ques
        self.__autopad_answer = autopad_answer
        
        self.vocab = set()
        self.word_counter = dict()
        self.word2id_dict = word2id_dict
        self.id2word_dict = id2word_dict
        
        self.context_ids = None
        self.context_sentences_ids = [] 
        self.question_ids = None
#        self.answer_ids = None
#         self.answer_masks = []
        self.answer_one_hot_labels = []

#         self.context_text = None
#         self.question_text = None
#         self.answer_text = None
        
#         self.num_tokens = None
#         self.max_sent_len_context = 0
#         self.max_num_sentences = 0
#         self.max_sent_len_question = 0
        
        self.__lower_words()
        self.__parse_data()
        self.__create_word2id_dict()
        self.__numericize_data()

    def __lower_words(self):
        self.context = [t.lower() for t in self.context]
        self.question = [t.lower() for t in self.question]
        self.answer = [t.lower() for t in self.answer]
    
    def __update_word_counter(self, sequence):
        """ Update word_counter with counts for words in a sentence
        
        Args:
            sequence (list<str>) : list of words in a sequence
        
        """
        for word in sequence:
            self.word_counter[word] = self.word_counter.get(word, 0) + 1
            
    def __create_vocab(self):
        """ Create set of most frequent unique words found in the training data """
        
        if self.max_vocab_size == None:
            self.vocab = set(self.word_counter.keys())
        else:
            self.vocab = set(sorted(self.word_counter, key=self.word_counter.get, reverse=True)[:self.max_vocab_size])
        
    def __shuffle_data(self, data):
        random.shuffle(data)
        return list(zip(*data))
    
    def __parse_data(self):
        
        for idx in range(len(self.context)):
            self.__update_word_counter(self.context[idx].split())
            self.__update_word_counter(self.question[idx].split())
            self.__update_word_counter(self.answer[idx].split())
        
        self.__create_vocab()
        
        shuffle = list(zip(self.context, self.question, self.answer, self.span))        
        self.context, self.question, self.answer, self.span = self.__shuffle_data(shuffle)
    
    def __create_word2id_dict(self):
        
        if self.word2id_dict == None:
            self.word2id_dict = dict()
            self.id2word_dict = dict()
            misc_tokens = ['PAD', 'UNK']

            for i, token in enumerate(misc_tokens):
                self.word2id_dict[token] = i

            for word in self.vocab:
                self.word2id_dict[word] = len(self.word2id_dict)

            self.vocab |= set(misc_tokens)
            self.id2word_dict = dict(zip(self.word2id_dict.values(), self.word2id_dict.keys()))
            self.num_tokens = len(self.word2id_dict)
    
    def __convert_word2id(self, word):
        try:
            word_id = self.word2id_dict[word]
        except:
            word_id = self.word2id_dict['UNK']
        return word_id
    
    def __apply_padding(self, s, pad_len):
        sequence = s[:]
        if len(sequence) < pad_len:
            sequence += [self.word2id_dict['PAD'] for i in range(pad_len - len(sequence))]
        elif len(sequence) > pad_len:
            sequence = sequence[:pad_len]
        else:
            pass
        return sequence
        
    def __get_seq_length_stats(self, sequences):
        max_len = 0
        min_len = 100000
        avg_len = 0
        for sequence in sequences:
            max_len = max(max_len, len(sequence))
            min_len = min(min_len, len(sequence))
            avg_len += len(sequence)
        avg_len = int(float(avg_len) / len(sequences))
        return min_len, max_len, avg_len

    def __get_max_sequence_lengths(self, context_ids, question_ids, answer_ids):
        min_context_len, max_context_len, avg_context_len = self.__get_seq_length_stats(context_ids)
        min_ques_len, max_ques_len, avg_ques_len = self.__get_seq_length_stats(question_ids)
        min_answer_len, max_answer_len, avg_answer_len = self.__get_seq_length_stats(answer_ids)

        if self.context_pad_len == None:
            if self.__autopad_context != None:
                if self.__autopad_context == 'min':
                    self.context_pad_len = min_context_len
                elif self.__autopad_context == 'max':
                    self.context_pad_len = max_context_len
                elif self.__autopad_context == 'avg':
                    self.context_pad_len = avg_context_len
            else:
                self.context_pad_len = avg_context_len
                
        if self.ques_pad_len == None:
            if self.__autopad_ques != None:
                if self.__autopad_ques == 'min':
                    self.ques_pad_len = min_ques_len
                elif self.__autopad_ques == 'max':
                    self.ques_pad_len = max_ques_len
                elif self.__autopad_ques == 'avg':
                    self.ques_pad_len = avg_ques_len
            else:
                self.ques_pad_len = avg_ques_len
                
        if self.answer_pad_len == None:
            if self.__autopad_answer != None:
                if self.__autopad_answer == 'min':
                    self.answer_pad_len = min_answer_len
                elif self.__autopad_answer == 'max':
                    self.answer_pad_len = max_answer_len
                elif self.__autopad_answer == 'avg':
                    self.answer_pad_len = avg_answer_len
            else:
                self.answer_pad_len = avg_answer_len 
                
        #print('Context length stats: min = {}, max = {}, avg = {}'.format(min_context_len, max_context_len, avg_context_len))
        #print('Context pad length set to: {}'.format(self.context_pad_len))
        
        #print('Question length stats: min = {}, max = {}, avg = {}'.format(min_ques_len, max_ques_len, avg_ques_len))
        #print('Question pad length set to: {}'.format(self.ques_pad_len))        
        
        #print('Answer length stats: min = {}, max = {}, avg = {}'.format(min_answer_len, max_answer_len, avg_answer_len))
        #print('Answer pad length set to: {}'.format(self.answer_pad_len))        
    def __convert_to_one_hot(self, span_ids, num_classes):
        label = []
        for idx in span_ids:
            one_hot = [0 for i in range(num_classes)]
            if idx >= num_classes - 1:
                one_hot[-1] = 1
            else:
                one_hot[idx] = 1
            label.append(one_hot)
        return label
    
    def __convert_to_mask(self, i, a):
        mask = (np.in1d(np.array(self.context_ids[i]), a[i]))*1
        if len(a[i])>1:
            count = 0
            while count < len(mask):
                if mask[count:count+len(a[i])].all():
                    count+=len(a[i])
                else:
                    mask[count] = False
                    count +=1
        return mask
        
    def __tokenize_sentences(self, context_ids, question_ids, answer_ids):
        """ Tokenizes sentences.
        :param raw: dict returned from load_babi
        :param word_table: WordTable
        :return:
        """
        context_tokens = []
        question_tokens = []
        #answer_tokens = []
        
        for i in range(len(context_ids)):
            c_tkn_ids = self.__apply_padding(context_ids[i], self.context_pad_len)
            context_tokens.append(c_tkn_ids)
            q_tkn_ids = self.__apply_padding(question_ids[i], self.ques_pad_len)
            question_tokens.append(q_tkn_ids)
            
            span_ids = [int(x) for x in self.span[i].split()]
            label = self.__convert_to_one_hot(span_ids, self.context_pad_len)
            self.answer_one_hot_labels.append(label)
            
#             if i%100 == 0:
#                 print(i)
            #a_tkn_ids = self.__apply_padding(answer_ids[i], self.answer_pad_len)
            #answer_tokens.append(a_tkn_ids)
        return context_tokens, question_tokens#, answer_tokens
        
    def __convert_text2ids(self, context, question, answer):
        """ Tokenizes sentences.
        :param raw: dict returned from load_babi
        :param word_table: WordTable
        :return:
        """
        context_ids = []
        question_ids = []
        answer_ids = []
        for i in range(len(context)):
            c_ids = [self.__convert_word2id(word) for word in context[i].split()]
            context_ids.append(c_ids)
            q_ids = [self.__convert_word2id(word) for word in question[i].split()]
            question_ids.append(q_ids)
            a_ids = [self.__convert_word2id(word) for word in answer[i].split()]
            answer_ids.append(a_ids)            
        return context_ids, question_ids, answer_ids
          
    def __convert_text2words(self, context, question, answer):
        context_words = []
        question_words = []
        answer_words = []
        for i in range(len(context)):
            c_words = context[i].split()
            context_words.append(c_words)
            q_words = question[i].split()
            question_words.append(q_words)
            a_words = answer[i].split()
            answer_words.append(a_words)
        return context_words, question_words, answer_words    

    def __get_max_sentence_length(self, stories):
        num_sentences = 0
        sen_len = 0
        for story in stories: 
            sentences = sent_tokenize(story)
            num_sentences = max(num_sentences, len(sentences))
            word_count = lambda sentence: len(word_tokenize(sentence))
            sen_len = max(sen_len, len((max(sentences, key=word_count)).split()))
        return sen_len, num_sentences
        
    def __context2sentences(self, story):
        story_sentences = []
        curr_sentence = []
        for word_id in story:
            if self.id2word_dict[word_id] == '.':
                if len(curr_sentence) < self.max_sent_len_context:
                    curr_sentence += [0 for i in range(self.max_sent_len_context - len(curr_sentence))]
                elif len(curr_sentence) > self.max_sent_len_context:
                    curr_sentence = curr_sentence[:self.max_sent_len_context]
                story_sentences.append(curr_sentence)
                curr_sentence = []
            else:
                curr_sentence.append(word_id)
            
        return story_sentences
    
    def __numericize_data(self):
        c, q, a = self.__convert_text2ids(self.context, self.question, self.answer)
        
        self.__get_max_sequence_lengths(c, q, a)
        
#         self.context_ids, self.question_ids, self.answer_ids = self.__tokenize_sentences(c, q, a)
        self.context_ids, self.question_ids = self.__tokenize_sentences(c, q, a)
            
#         self.context_text, self.question_text, self.answer_text = self.__convert_text2words(self.context,
#                                                                                             self.question,
#                                                                                             self.answer)
        #self.max_sent_len_context, self.max_num_sentences = self.__get_max_sentence_length(self.context)
        #print("max_sent_len_context: {} max_num_sentences: {}".format(self.max_sent_len_context, self.max_num_sentences))
        #self.max_sent_len_question, ques_num_sentences = self.__get_max_sentence_length(self.question)
        #print("max_sent_len_ques: {} ques_num_sentences: {}".format(self.max_sent_len_question, ques_num_sentences))
        #for c_ids in c:
            #self.context_sentences_ids.append(self.__context2sentences(c_ids))

In [8]:
UNKNOWN = "*UNKNOWN*"

def generate_embedding_files(filename):
    embeddings = {}
    for line in open(filename):
        parts = line.split()
        embeddings[parts[0]] = list(map(float, parts[1:]))
    embedding_size = len(list(embeddings.values())[0])
    embeddings[UNKNOWN] = [0.0 for _ in range(embedding_size)]
    
    words = embeddings.keys()
    embedding_matrix = np.array([embeddings[word] for word in list(embeddings.keys())])
    return words, embedding_matrix

glove_words, glove_embedding_matrix = generate_embedding_files('/media/ai2-rey/data_disk/data_sets/glove.6B/glove.6B.50d.txt')
glove_vocab_lookup = {word: i for i, word  in enumerate(glove_words)}
glove_vocab_size, glove_embedding_size= glove_embedding_matrix.shape

In [9]:
class DataIterator:  
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
        self.data_iterator = self.make_random_iter()
        
    def next_batch(self):
        try:
            idxs = next(self.data_iterator)
        except StopIteration:
            self.data_iterator = self.make_random_iter()
            idxs = next(self.data_iterator)
            
        batch = [self.data[i] for i in idxs]
        batch_idxs = [idx for idx in idxs]
        return batch, batch_idxs

    def make_random_iter(self):
        splits = np.arange(self.batch_size, len(self.data), self.batch_size)
        it = np.split(np.random.permutation(range(len(self.data))), splits)[:-1]
        return iter(it)

In [10]:
train = QA_Dataset(train_cont, train_ques, train_ans, train_span,
                   context_pad_len = 200, ques_pad_len = 20,
                   autopad_context='max', autopad_ques = 'max', autopad_answer='max')

In [11]:
val = QA_Dataset(val_cont, val_ques, val_ans, val_span,
                 train.word2id_dict, train.id2word_dict,
                 context_pad_len=train.context_pad_len, 
                 ques_pad_len = train.ques_pad_len, 
                 answer_pad_len=train.answer_pad_len,
                 autopad_context='max', autopad_ques='max', autopad_answer='max')

test = QA_Dataset(test_cont, test_ques, test_ans, test_span,
                  train.word2id_dict, train.id2word_dict,
                  context_pad_len=train.context_pad_len, 
                  ques_pad_len = train.ques_pad_len,
                  answer_pad_len=train.answer_pad_len,
                  autopad_context='max', autopad_ques='max', autopad_answer='max')

In [12]:
del train_cont, train_ques, train_ans, train_span
del val_cont, val_ques, val_ans, val_span
del test_cont, test_ques, test_ans, test_span

In [13]:
train_ids = list(zip(train.context_ids, train.question_ids, train.answer_one_hot_labels))
val_ids = list(zip(val.context_ids,val.question_ids,val.answer_one_hot_labels))
test_ids = list(zip(test.context_ids,test.question_ids, test.answer_one_hot_labels))

In [14]:
train_data_iter = DataIterator(train_ids,128)
val_data_iter = DataIterator(val_ids,128)
test_data_iter = DataIterator(test_ids,128)
#deploy_data_iter = DataIterator(train_ids,1)

In [17]:
train.ques_pad_len

20

In [18]:
train.context_pad_len

200

In [19]:
def prepare_model_dir():
    model_dir = os.getcwd() + '/'
    if not os.path.exists(model_dir + 'weights'):
        os.makedirs(model_dir + 'weights')
    return model_dir

In [20]:
def write_to_log(string,filename):
    print(string)
    with open(filename,'a') as write_file:
        write_file.write(string + '\n')

In [21]:
def convert_bounds_to_words(bounds, passage):
    id_span = list(range(bounds[0], bounds[1]+1))
    return ' '.join([train.id2word_dict[passage[idx]] for idx in id_span])

In [22]:
def normalize_answer(s):
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    
    def white_space_fix(text):
        return ' '.join(text.split())
    
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    
    def lower(text):
        return text.lower()
    
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [23]:
def exact_match_score(prediction, ground_truth):
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

In [25]:
passage_max_len = train.context_pad_len
question_max_len = train.ques_pad_len
#output of lstm is n_steps or passage length by hidden size 
n_hidden = 150 
num_indices = 2
#batch_size = 128
n_hidden_layers = 1
learning_rate = 0.001
blend_units = 6
train_iters = 100000
keep_prob = 0.5
display_step = 200
val_interval = 1000
save_weights_interval = 1000
word_dim = 50

In [26]:
model_dir = prepare_model_dir()
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [27]:
passage = tf.placeholder(tf.int32, [None, passage_max_len], name='passage')
question = tf.placeholder(tf.int32,[None, question_max_len], name='question')
y = tf.placeholder(tf.float32,[None,2,passage_max_len])

dropout_keep_prob = tf.placeholder(tf.float32)
batch_size = tf.placeholder(tf.int32)
p_seq_len = tf.placeholder(tf.int32)
q_seq_len = tf.placeholder(tf.int32)

embeddings = tf.constant(glove_embedding_matrix, name='embeddings', dtype=tf.float32)

#preprocessing layer 
passage_embedded = tf.nn.embedding_lookup(embeddings, passage) #P = batch_size(b), word_dims(d), max_passage_len(P)
question_embedded = tf.nn.embedding_lookup(embeddings, question)
print(passage_embedded[0])
print(passage_embedded)

with tf.variable_scope('passage_lstm'):
    passage_cell_fwd = tf.nn.rnn_cell.LSTMCell(n_hidden)
    passage_cell_fwd = tf.nn.rnn_cell.DropoutWrapper(passage_cell_fwd, output_keep_prob=dropout_keep_prob)
    passage_cell_fwd = tf.nn.rnn_cell.MultiRNNCell([passage_cell_fwd]*n_hidden_layers)
    
    passage_cell_bwd = tf.nn.rnn_cell.LSTMCell(n_hidden)
    passage_cell_bwd = tf.nn.rnn_cell.DropoutWrapper(passage_cell_bwd, output_keep_prob=dropout_keep_prob)
    passage_cell_bwd = tf.nn.rnn_cell.MultiRNNCell([passage_cell_bwd]*n_hidden_layers)
    #H_p shape = batch_size, max_passage_len, n_hidden
    
    H_p, _ = tf.nn.bidirectional_dynamic_rnn(passage_cell_fwd, passage_cell_bwd, passage_embedded, p_seq_len,dtype=tf.float32)
    print(H_p)
    H_p = tf.concat(2,H_p) #(batch_size,passage_len,n_hidden)*2 in a tuple
    print(H_p)
    
with tf.variable_scope('question_lstm'):
    question_cell_fwd = tf.nn.rnn_cell.LSTMCell(n_hidden)
    question_cell_fwd = tf.nn.rnn_cell.DropoutWrapper(question_cell_fwd, output_keep_prob=dropout_keep_prob)
    question_cell_fwd = tf.nn.rnn_cell.MultiRNNCell([question_cell_fwd]*n_hidden_layers)
    
    question_cell_bwd = tf.nn.rnn_cell.LSTMCell(n_hidden)
    question_cell_bwd = tf.nn.rnn_cell.DropoutWrapper(question_cell_bwd, output_keep_prob=dropout_keep_prob)
    question_cell_bwd = tf.nn.rnn_cell.MultiRNNCell([question_cell_bwd]*n_hidden_layers)
    
    H_q, _ = tf.nn.bidirectional_dynamic_rnn(question_cell_fwd, question_cell_bwd, question_embedded, q_seq_len, dtype=tf.float32)
    print(H_q)
    H_q = tf.concat(2,H_q)
    print(H_q)

#Match LSTM layer 
#Weights and bias to compute G
W_q = tf.Variable(tf.truncated_normal(shape=[2*n_hidden, n_hidden], stddev=0.1))
W_p = tf.Variable(tf.truncated_normal(shape=[2*n_hidden, n_hidden], stddev=0.1))
W_r = tf.Variable(tf.truncated_normal(shape=[n_hidden, n_hidden], stddev=0.1))
b_p = tf.Variable(tf.constant(0.1, shape=[n_hidden]))

#Weight and bias to compute a
w= tf.Variable(tf.truncated_normal(shape=[n_hidden,1], stddev=0.1))#unsure 
#b_alpha shape = 1,none
b_alpha = tf.Variable(tf.constant(0.1, shape = [1]))

Tensor("strided_slice:0", shape=(200, 50), dtype=float32)
Tensor("embedding_lookup:0", shape=(?, 200, 50), dtype=float32)
(<tf.Tensor 'passage_lstm/BiRNN/FW/FW/transpose:0' shape=(?, 200, 150) dtype=float32>, <tf.Tensor 'passage_lstm/ReverseSequence:0' shape=(?, 200, 150) dtype=float32>)
Tensor("passage_lstm/concat:0", shape=(?, 200, 300), dtype=float32)
(<tf.Tensor 'question_lstm/BiRNN/FW/FW/transpose:0' shape=(?, 20, 150) dtype=float32>, <tf.Tensor 'question_lstm/ReverseSequence:0' shape=(?, 20, 150) dtype=float32>)
Tensor("question_lstm/concat:0", shape=(?, 20, 300), dtype=float32)


In [28]:
H_q = tf.reshape(H_q, [-1, 2*n_hidden])
WH_q = tf.matmul(H_q, W_q)
print(WH_q)
WH_q = tf.reshape(WH_q,[-1, question_max_len, n_hidden])
print(WH_q)

Tensor("MatMul:0", shape=(?, 150), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 20, 150), dtype=float32)


In [29]:
H_r_forward = []
H_r_backward = []

with tf.variable_scope('forward_match_lstm'):
    forward_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(n_hidden), output_keep_prob=dropout_keep_prob)
    forward_state = forward_cell.zero_state(batch_size, dtype=tf.float32)
    h = forward_state.h #h shape =(batch_size, n_hidden)

    for i in range(passage_max_len):
        H_p_i =tf.squeeze(tf.slice(H_p, [0,i,0], [-1,1,-1]),1)
        WH_p_i = tf.matmul(H_p_i, W_p)
        
        W_rh = tf.matmul(h,W_r)
        
        G_tile = tf.tile((WH_p_i + W_rh + b_p), [question_max_len, 1])
        G_tile = tf.reshape(G_tile, [-1, question_max_len, n_hidden])
        G_forward = tf.tanh(WH_q + G_tile)
        
        G_forward = tf.reshape(G_forward, [-1,n_hidden])
        wG_forward = tf.matmul(G_forward,w)
        a_tile = tf.tile(b_alpha, [question_max_len])
        wG_forward = tf.reshape(tf.squeeze(wG_forward,1),[-1, question_max_len])
        alpha_forward = tf.nn.softmax(wG_forward + a_tile)

        H_q = tf.reshape(H_q,[-1, 2*n_hidden, question_max_len])
        alpha_forward = tf.expand_dims(alpha_forward, 2)
        H_q_alpha_forward = tf.batch_matmul(H_q, alpha_forward)
        H_q_alpha_forward = tf.squeeze(H_q_alpha_forward, 2)
        
        z_forward = tf.concat(1, [H_p_i, H_q_alpha_forward])

        h,forward_state = forward_cell(z_forward, forward_state)
        tf.get_variable_scope().reuse_variables()
        H_r_forward.append(h)
        if i%100 == 0:
            print(i)

    H_r_forward = tf.transpose(tf.stack(H_r_forward),[1,0,2])

0
100


In [30]:
with tf.variable_scope('backward_match_lstm'):
    backward_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(n_hidden), output_keep_prob=dropout_keep_prob)
    backward_state = backward_cell.zero_state(batch_size, dtype=tf.float32)
    h = backward_state.h #h shape =(bathc_size, n_hidden)

    for i in reversed(range(passage_max_len)):
        H_p_i =tf.squeeze(tf.slice(H_p, [0,i,0], [-1,1,-1]),1)
        WH_p_i = tf.matmul(H_p_i, W_p)
        
        W_rh = tf.matmul(h,W_r)
        
        G_tile = tf.tile((WH_p_i + W_rh + b_p), [question_max_len, 1])
        G_tile = tf.reshape(G_tile, [-1, question_max_len, n_hidden])
        G_backward = tf.tanh(WH_q + G_tile)
        
        G_backward = tf.reshape(G_backward, [-1,n_hidden])
        wG_backward = tf.matmul(G_backward,w)
        a_tile = tf.tile(b_alpha, [question_max_len])
        wG_backward = tf.reshape(tf.squeeze(wG_backward,1),[-1, question_max_len])
        alpha_backward = tf.nn.softmax(wG_backward + a_tile)

        H_q = tf.reshape(H_q,[-1, 2*n_hidden, question_max_len])
        alpha_backward = tf.expand_dims(alpha_backward, 2)
        H_q_alpha_backward = tf.batch_matmul(H_q, alpha_backward)
        H_q_alpha_backward = tf.squeeze(H_q_alpha_backward, 2)
        
        z_backward = tf.concat(1, [H_p_i, H_q_alpha_backward])

        h,backward_state = backward_cell(z_backward, backward_state)
        tf.get_variable_scope().reuse_variables()
        H_r_backward.append(h)
        if i%100 == 0:
            print(i)
            
    H_r_backward = list(reversed(H_r_backward))
    H_r_backward = tf.transpose(tf.stack(H_r_backward),[1,0,2])
    

100
0


In [31]:
H_r = tf.concat(2, [H_r_forward, H_r_backward])
print(H_r)

Tensor("concat:0", shape=(?, 200, 300), dtype=float32)


In [32]:
# H_r = H_r_forward
# H_r = tf.reshape(H_r,[-1,n_hidden])

In [33]:
#Answer Pointer Layer 

#Weights and bias to compute 'F'
V = tf.Variable(tf.truncated_normal(shape=[2*n_hidden, n_hidden], stddev=0.1))
print(V)
W_a = tf.Variable(tf.truncated_normal(shape=[n_hidden, n_hidden], stddev=0.1))
b_a = tf.Variable(tf.constant(0.1, shape=[n_hidden]))

#weight and bias to computer 'beta'
v = tf.Variable(tf.truncated_normal(shape=[n_hidden,1], stddev=0.1))
b_beta = tf.Variable(tf.constant(0.1, shape=[1]))

#Only calculate 'VH' once 
H_r = tf.reshape(H_r, [-1,2*n_hidden]) #shape = (2*n_hidden, batch&passage_max_len)
print(H_r)
VH = tf.matmul(H_r,V) #shape (hidden_size, passage_max_len)
VH = tf.reshape(VH,[-1, passage_max_len, n_hidden])

output_logits = []
output_probs = []

with tf.variable_scope('answer_pointer_lstm'):
    pointer_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(n_hidden), output_keep_prob=dropout_keep_prob)
    pointer_state = pointer_cell.zero_state(batch_size, dtype=tf.float32)
    h = pointer_state.h
    
    for k in range(2):
        W_ah = tf.matmul(h, W_a)
        
        F_tile = tf.tile(tf.expand_dims(tf.add(W_ah,b_a),1),[1,passage_max_len,1])
        F = tf.tanh(tf.add(VH, F_tile))
        F = tf.reshape(F, [-1,n_hidden])   
        vF = tf.matmul(F,v)
        beta_tile = tf.tile(b_beta, [passage_max_len])
        vF = tf.reshape(tf.squeeze(vF, 1), [-1,passage_max_len])
        beta_logits = tf.add(vF,beta_tile)
        beta = tf.nn.softmax(beta_logits)
        output_logits.append(beta_logits)
        output_probs.append(beta)


    #    beta = tf.transpose(beta,[1,0])
        beta = tf.expand_dims(beta,2) #unsure and changed 
        H_r = tf.reshape(H_r, [-1, 2*n_hidden, passage_max_len])
        H_r_beta = tf.batch_matmul(H_r, beta)
        H_r_beta = tf.squeeze(H_r_beta,2)
        h,pointer_state = pointer_cell(H_r_beta, pointer_state)
        tf.get_variable_scope().reuse_variables()
        print(k)

probs_start = output_probs[0]
probs_end = output_probs[1]
probs = tf.concat(1,[tf.expand_dims(probs_start,1), tf.expand_dims(probs_end,1)])
print(probs)

start_label, end_label = tf.split(1,2,y)
start_label = tf.squeeze(start_label,1)
end_label = tf.squeeze(end_label,1)
print(start_label)

combined_prob = tf.batch_matmul(tf.batch_matmul(tf.expand_dims(probs_start, 1), tf.expand_dims(start_label, 2)), 
                                tf.batch_matmul(tf.expand_dims(probs_end, 1), tf.expand_dims(end_label, 2)))
loss = -tf.reduce_mean(tf.log(combined_prob))
print(loss)

params = tf.trainable_variables()
gradients = tf.gradients(loss, params)
gradient_norm = tf.global_norm(gradients)
print(gradient_norm)

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
print(optimizer)

correct_start = tf.equal(tf.argmax(probs_start, axis=1), tf.argmax(start_label, axis=1))
correct_end = tf.equal(tf.argmax(probs_end, axis=1), tf.argmax(end_label, axis=1))
equal_vals = tf.equal(tf.stack([tf.argmax(probs_start, axis=1), tf.argmax(probs_end, axis=1)], axis=1),
                      tf.stack([tf.argmax(start_label, axis=1), tf.argmax(end_label, axis=1)], axis=1))
correct_prediction = tf.reduce_all(equal_vals, axis=1)
print(correct_prediction)

start_accuracy = tf.reduce_mean(tf.cast(correct_start, tf.float32))
end_accuracy = tf.reduce_mean(tf.cast(correct_end, tf.float32))
mean_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(mean_accuracy)


Tensor("Variable_6/read:0", shape=(300, 150), dtype=float32)
Tensor("Reshape_2:0", shape=(?, 300), dtype=float32)
0
1
Tensor("concat_1:0", shape=(?, 2, 200), dtype=float32)
Tensor("Squeeze:0", shape=(?, 200), dtype=float32)
Tensor("Neg:0", shape=(), dtype=float32)
Tensor("global_norm/global_norm:0", shape=(), dtype=float32)
name: "Adam"
op: "NoOp"
input: "^Adam/update_passage_lstm/BiRNN/FW/MultiRNNCell/Cell0/LSTMCell/W_0/ApplyAdam"
input: "^Adam/update_passage_lstm/BiRNN/FW/MultiRNNCell/Cell0/LSTMCell/B/ApplyAdam"
input: "^Adam/update_passage_lstm/BiRNN/BW/MultiRNNCell/Cell0/LSTMCell/W_0/ApplyAdam"
input: "^Adam/update_passage_lstm/BiRNN/BW/MultiRNNCell/Cell0/LSTMCell/B/ApplyAdam"
input: "^Adam/update_question_lstm/BiRNN/FW/MultiRNNCell/Cell0/LSTMCell/W_0/ApplyAdam"
input: "^Adam/update_question_lstm/BiRNN/FW/MultiRNNCell/Cell0/LSTMCell/B/ApplyAdam"
input: "^Adam/update_question_lstm/BiRNN/BW/MultiRNNCell/Cell0/LSTMCell/W_0/ApplyAdam"
input: "^Adam/update_question_lstm/BiRNN/BW/MultiRN

In [34]:
sess.run(tf.global_variables_initializer())

In [35]:
def evaluate(sess, feed_dict, passage, labels):
    model_pred = sess.run(probs, feed_dict= feed_dict)
    em_total = 0
    
    for i in range(len(model_pred)):
        label_ids = [np.argmax(x) for x in labels[i]]
        pred_ids = [np.argmax(x) for x in model_pred[i]]
        
        label_string = convert_bounds_to_words(label_ids, passage[i])
        pred_string = convert_bounds_to_words(pred_ids, passage[i])
        em_total += exact_match_score(pred_string, label_string)
        if i <5:
            print(i)
            print(label_string)
            print(i)
            print(pred_string)
    return em_total / float(len(model_pred)), pred_string, label_string

In [36]:
log_file = model_dir + 'Pnter_ntwrk_log.txt'
saver = tf.train.Saver(max_to_keep=100)
log = open(log_file,'w')
log.close()

In [37]:
for train_iter in range(train_iters):
    
    train_iter +=1
    train_batch, _ = train_data_iter.next_batch()
    train_context, train_question, train_answer = zip(*train_batch)
    if train_iter == 1:
        train_feed_dict = {passage: train_context,
                           question: train_question,
                           y: train_answer,
                           dropout_keep_prob: 1.0, 
                           p_seq_len:np.ones(train_data_iter.batch_size) * passage_max_len,
                           q_seq_len:np.ones(train_data_iter.batch_size) * question_max_len,
                           batch_size: train_data_iter.batch_size}
        
        training_loss, start_acc, end_acc, acc, grad_norm = tuple(sess.run([loss, start_accuracy, end_accuracy,
                                                                            mean_accuracy, gradient_norm],
                                                                  feed_dict=train_feed_dict))

        log_output= ("Iter {}, start_accuracy = {:.4f}, end_accuracy = {:.4f}, accuracy = {:.4f}, train loss = {:.6f}"
                     .format(train_iter, start_acc, end_acc, acc, training_loss))
        print(log_output)
        em_score, _, _ =evaluate(sess, train_feed_dict, train_context, train_answer)
        
        write_to_log(log_output, log_file)
        write_to_log('Exact Match Score: {:.4f}'.format(em_score), log_file)
        
    sess.run(optimizer, feed_dict={passage: train_context,
                                   question: train_question, 
                                   y: train_answer, 
                                   dropout_keep_prob: keep_prob, 
                                   p_seq_len:np.ones(train_data_iter.batch_size) * passage_max_len,
                                   q_seq_len:np.ones(train_data_iter.batch_size) * question_max_len,
                                   batch_size: train_data_iter.batch_size})
    if train_iter % display_step == 0:
        disp_feed_dict = {passage: train_context, 
                          question: train_question, 
                          y: train_answer,
                          dropout_keep_prob: 1.0,
                          p_seq_len:np.ones(train_data_iter.batch_size) * passage_max_len,
                          q_seq_len:np.ones(train_data_iter.batch_size) * question_max_len,
                          batch_size: train_data_iter.batch_size
                         }
        training_loss, start_acc, end_acc, acc= tuple(sess.run([loss, start_accuracy, end_accuracy, 
                                                                mean_accuracy],
                                                               feed_dict=disp_feed_dict))
        log_output = ("Iter {}, start_accuracy = {:.4f}, end_accuracy = {:.4f}, accuracy = {:.4f}, train loss = {:.6f}"
              .format(train_iter, start_acc, end_acc, acc, training_loss))
        print(log_output)
        em_score, _, _ = evaluate(sess, disp_feed_dict, train_context, train_answer)
        write_to_log(log_output, log_file)
        write_to_log('Exact Match Score: {:.4f}'.format(em_score), log_file)
        
    if val_data_iter !=None and train_iter%val_interval ==0:
        val_batch, _ = val_data_iter.next_batch()
        val_context, val_question, val_answer = zip(*val_batch)
        val_feed_dict = {passage: val_context,
                         question: val_question, 
                         y: val_answer,
                         dropout_keep_prob: 1.0,
                         p_seq_len:np.ones(val_data_iter.batch_size) * passage_max_len,
                         q_seq_len:np.ones(val_data_iter.batch_size) * question_max_len,
                         batch_size: val_data_iter.batch_size}
        
        val_loss, val_start_acc, val_end_acc, val_acc = tuple(sess.run([loss, start_accuracy, end_accuracy, 
                                                       mean_accuracy],
                                                      feed_dict=val_feed_dict))
        
        log_output = ("Iter {}, val_start_accuracy = {:.4f}, val_end_acc = {:.4f}, val_acc = {:.4f}, val loss = {:.6f}"
                      .format(train_iter, val_start_acc, val_end_acc, val_acc, val_loss))
        print(log_output)
        em_score, _, _ = evaluate(sess, val_feed_dict, val_context, val_answer)
        write_to_log(log_output, log_file)
        write_to_log('Exact Match Score: {:.4f}'.format(em_score), log_file)
    
    if train_iter%save_weights_interval ==0:
        weights_dir = model_dir  + "weights/PntrNtwrk_weights_iter--{}.ckpt".format(train_iter)
        save_path = saver.save(sess, weights_dir)
        save_string = "Model saved in file: {}".format(save_path)
        print(save_string)
        write_to_log(save_string, log_file)
    

Iter 1, start_accuracy = 0.0000, end_accuracy = 0.0000, accuracy = 0.0000, train loss = 10.414585
0
over three thousand
0
by
1
western europe
1
satellite
2
the state security administration
2
.
3
interfere with commuter and troop travel
3
world
4
djibouti
4
10,000
Iter 1, start_accuracy = 0.0000, end_accuracy = 0.0000, accuracy = 0.0000, train loss = 10.414585
Exact Match Score: 0.0000
Iter 200, start_accuracy = 0.0312, end_accuracy = 0.0469, accuracy = 0.0234, train loss = 8.852318
0
scholarly tradition of the greek academies was maintained during roman times with several academic institutions
0
paideia
1
shenzhen stock exchange
1
china
2
separate from ( but in contact with ) each other
2
from
3
medieval to modern
3
1808
4
dame lois browne-evans , and her trinidadian-born husband , john evans
4
made
Iter 200, start_accuracy = 0.0312, end_accuracy = 0.0469, accuracy = 0.0234, train loss = 8.852318
Exact Match Score: 0.0234
Iter 400, start_accuracy = 0.1016, end_accuracy = 0.1094, accur

KeyboardInterrupt: 

In [ ]:
.

128, 766, 1